In [ ]:
!pip install neo4j
!pip install graphdatascience

In [ ]:
import logging
import sys

from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable

## Establish Neo4j connection

In [ ]:
# ## Credentials - Sheetal
# NEO4J_URI="neo4j+s://046bad63.databases.neo4j.io"
# NEO4J_USERNAME="neo4j"
# NEO4J_PASSWORD="I13rxtvRV7ki8rIwqnQ69SXGCW6C5LXslL6yzJZ20XA"
# AURA_INSTANCENAME="Instance01"

## Credentials - local instance (neo4j desktop)
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="dsc202"

# Connection
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [ ]:
session = driver.session()

## Importing Nodes from CSV

In [ ]:
q1 = '''
LOAD CSV WITH HEADERS FROM "file:///Industries.csv" AS row
MERGE (i:Industry {industryId:row.title})
RETURN count(i);
'''
session.execute_write(q1)

In [ ]:
q2 = '''
LOAD CSV WITH HEADERS FROM "file:///Assignees.csv" AS row
MERGE (a:Assignee {assigneeName:row.current_assignee})
RETURN count(a);
'''
session.execute_write(q2)

In [ ]:
q3 = '''
LOAD CSV WITH HEADERS FROM "file:///Patents.csv" AS row
MERGE (p:Patent {patentId:row.patentid})
SET p.inventors = row.inventor_list
RETURN count(c);
'''
session.execute_write(q3)

In [ ]:
q4 = '''
LOAD CSV WITH HEADERS FROM "file:///Countries.csv" AS row
MERGE (c:Country {countryId:row.countrycode})
SET c.countryName = row.countryname
RETURN count(c);
'''
session.execute_write(q4)

## Importing relationships from CSV

In [ ]:
q5 = '''
LOAD CSV WITH HEADERS FROM "file:///Patent_Assignee.csv" AS row
MATCH (p:Patent {patentId:row.patentid}), (a:Assignee {assigneeName:row.assignee_name_current})
MERGE (p)-[rel:ASSIGNED_TO]->(a)
RETURN count(rel);
'''
session.execute_write(q5)

In [ ]:
q6 = '''
LOAD CSV WITH HEADERS FROM "file:///Patent_Industry.csv" AS row
MATCH (p:Patent {patentId:row.patentid}), (i:Industry {industryId:row.clean_industry})
MERGE (p)-[rel:BELONGS_TO]->(i)
RETURN count(rel);
'''
session.execute_write(q6)

In [ ]:
q7 = '''
LOAD CSV WITH HEADERS FROM "file:///Assignee_Industry.csv" AS row
MATCH (a:Assignee {assigneeName:row.title}), (i:Industry {industryId:row.industry})
MERGE (a)-[rel:BELONGS_TO]->(i)
RETURN count(rel);
'''
session.execute_write(q7)

In [ ]:
q8 = '''
LOAD CSV WITH HEADERS FROM "file:///Patent_Country.csv" AS row
MATCH (p:Patent {patentId:row.patentid}), (c:Country {countryId:row.countrycode})
MERGE (p)-[r:VALID_IN]->(c)
RETURN count(r);
'''
session.execute_write(q8)

In [ ]:
q9 = '''
LOAD CSV WITH HEADERS FROM "file:///PatentIssuingCouuntryCode.csv" AS row
MATCH (p:Patent {patentId:row.patentid}), (c:Country {countryId:row.countrycode})
MERGE (p)-[r:ISSUED_BY]->(c)
RETURN count(r);
'''
session.execute_write(q9)

## Neo4j test query

In [ ]:
q10 = '''MATCH (a:Assignee)-[r:BELONGS_TO]->(i:Industry)
WITH a, i, size((a)-->()) AS outgoingCount
ORDER BY outgoingCount DESC
RETURN DISTINCT a, i, outgoingCount
LIMIT 100000;'''

result = session.run(q10)
result.data()

In [ ]:
session.close()
driver.close()